# Tree Search Algorithms

In this notebook we will implement some of the tree search algorithms that are discussed in Russell and Norvig. 

I would initially like to implement the following uniformed algorithms, which do not rely on any heuristics:

- [x] Breadth-First Search 
- [x] Depth-First Search
- [x] Uniform-Cost Search

Here I will redefine the Node classes, rather than take them from `node.py`. This should be a much more minimal implementation. 

### Imports

In [1]:
from rl4uc.environment import make_env

import numpy as np
import pandas as pd
import itertools
import copy
import queue

### Node class

R&N say that a node needs the following attributes:

- State: the state of the environment that the node corresponds to
- Parent: the parent node in the search tree
- Action: the action used to get from parent to the current node
- Path cost: the cost of reaching the current node

In our case, we will consider that there are only unique paths to each node. 

*Note: `env` and `state` are used somewhat interchangeably in these functions. An env object represents a state, but also includes the transition dynamics. Hence, we will usually pass `env` as an argument rather than state.*


In [2]:
class Node(object):
    def __init__(self, env, parent, action, action_id, path_cost):
        self.state = env
        self.parent = parent
        self.action = action
        self.action_id = action_id
        self.path_cost = path_cost
        self.is_expanded = False

In addition, we will need some other general functions for our tree search algorithms. 

The first is `get_actions(env)` which returns the set of applicable actions from `env`. *Note: this function is where any guided tree search should be implemented. If we have an expansion policy, it can be used here to reduce the set of applicable actions.*

In [3]:
def get_actions(env):
    """Get all applicable actions from the `env` state"""
    constrained_gens = np.where(np.logical_or(env.must_on, env.must_off))
    unconstrained_gens = np.delete(np.arange(env.num_gen), constrained_gens)

    # All permutations of available generators
    all_perms = np.array(list(itertools.product(range(2), repeat=unconstrained_gens.size)))

    # Create action array 
    actions = np.zeros((all_perms.shape[0], env.num_gen))
    actions[:,constrained_gens] = env.commitment[constrained_gens]
    actions[:,unconstrained_gens] = all_perms
    
    return actions

Next we need a function for generating a child node. This takes a node as input, and outputs a child node with `node` as its parent. *Note: this is where, for stochastic reward functions, we may want to try and estimate the expected reward by considering possible scenarios.*

In [4]:
def get_child_node(node, action, deterministic=True):
    """
    Return a child node corresponding to taking `action` from the state 
    corresponding to `node`.
    
    The child node has `node` as its parent.
    """
    action_id = ''.join(str(int(i)) for i in action)
    action_id = int(action_id, 2)

    new_env = copy.deepcopy(node.state)
    _, reward, _ = new_env.step(action, deterministic=deterministic)
    cost = -reward

    child = Node(env=new_env,
                parent=node,
                action=action, 
                action_id=action_id,
                path_cost = node.path_cost + cost)
    
    return child

After we have found our final leaf node, we need to get out the solution. This recursively traverses the tree to the leaf, and returns the list of actions of the solution path.

In [5]:
def get_solution(node):
    """Return the solution path (list of actions) leading to node."""
    s = []
    path_cost = node.path_cost
    while node.parent is not None:
        s.insert(0, node.action)
        node = node.parent
    return s, path_cost

Finally, let's implement a function for testing any version. It should return the solution path and the path cost. 

In [15]:
def test(tree_search_func,
         num_gen=5,
         problem_fn='../data/day_ahead/5gen/30min/profile_2017-06-25.csv',
         periods=3):
    """Test a tree search algorithm."""
    df = pd.read_csv(problem_fn)[:periods]
    env = make_env(mode='test', profiles_df=df, num_gen=num_gen)
    env.reset()
    solution = tree_search_func(env)
    return solution
    

### General Tree Search Algorithm

Let's begin by defining a general tree search algorithm, as in R&N (Figure 3.7).

The general tree search algorithm looks something like this:

`Initialise the frontier as a queue
Loop:
  if the frontier is empty: return failure
  choose a leaf node from the frontier (and remove it)
  if the node contains a goal state then return the corresponding solution
  expand the chosen node and add the results to the frontier
`

Note that since we are dealing with trees not graphs, we do not need to keep track of explored nodes. This simplifies things in terms of memory requirements, as we only need to keep track of the frontier. 

### Breadth-First Search

The first algorithm we will look at is breadth-first search (BFS).

R&N: 

> BFS is a simple strategy in which the root node is expanded first, then all the successors of the root node are expanded next, then *their* successors, and so on.*

BFS uses a first-in, first-out (FIFO) queue for the frontier and prioritises the shallowest nodes to expand. It therefore finds the quickest route to the bottom of the tree, and will find the optimal path if all step costs are uniform.

In [16]:
def bfs(env):
    """Breadth first search"""
    node = Node(env=env,
                parent=None,
                action=None,
                action_id=None,
                path_cost=0)
    if node.state.is_terminal():
        return get_solution(node)
    frontier = queue.SimpleQueue() # FIFO 
    frontier.put(node)
    while True:
        assert frontier, "Failed to find a goal state"
        node = frontier.get()
        for action in get_actions(node.state):
            child = get_child_node(node, action)
            if child.state.is_terminal():
                return get_solution(child)
            frontier.put(child)
            

In [17]:
%%time
path, path_cost = test(bfs)
print("Path: {}".format(path))
print("Cost: {}".format(path_cost))


Path: [array([0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.])]
Cost: 5669431.643323701
CPU times: user 231 ms, sys: 8.01 ms, total: 239 ms
Wall time: 255 ms


While I have chosen to include this algorithm for completeness, it doesn't serve our purposes at all. A goal state is any terminal state in our problem definition. As a result, BFS will just find the shallowest node, irrespective of cost. Note that the `path_cost` doesn't appear at all in the algorithm. We can verify this by seeing that we end up with a bogus solution: keep everything off. This is chosen as it is the first action that is generated in `get_child_node()`. 

### Depth-First Search

Depth-first search always chooses the deepest node to expand. Its implementation is exactly the same as BFS, except it uses a last-in, first-out queue (LIFO). 

In [18]:
def dfs(env):
    """Breadth first search"""
    node = Node(env=env,
                parent=None,
                action=None,
                action_id=None,
                path_cost=0)
    if node.state.is_terminal():
        return get_solution(node)
    frontier = queue.LifoQueue()
    frontier.put(node)
    while True:
        assert frontier, "Failed to find a goal state"
        node = frontier.get()
        for action in get_actions(node.state):
            child = get_child_node(node, action)
            if child.state.is_terminal():
                return get_solution(child)
            frontier.put(child)

In [19]:
%%time
path, path_cost = test(dfs)
print("Path: {}".format(path))
print("Cost: {}".format(path_cost))

Path: [array([1., 1., 1., 1., 1.]), array([1., 1., 1., 1., 1.]), array([0., 1., 1., 1., 0.])]
Cost: 17115.763001348703
CPU times: user 40.7 ms, sys: 4.49 ms, total: 45.2 ms
Wall time: 43 ms


Much like BFS, this search does not consider the path cost at all and is not optimal.

### Uniform-Cost Search

We will now look at a simple tree search algorithm which does consider the path cost. Uniform cost search stores the frontier as a *priority queue*. The priority queue orders nodes in the frontier by their path cost. When we take a node from the frontier, we always take the lowest cost one.

R&N also notes another significant differences between BFS and uniform-cost search: 

- The goal test is applied to a node when it is selected for expansion, rather than when it is first generated). This prevents the algorithm terminating on the first goal state it finds, which could be on a suboptimal path. This modification thus ensures optimality. 

*Note: one other difference is also mentioned, which refers to the case where the state already appears in the frontier with higher path cost. However, this is not applicable to our problem, where the states only ever appear once on the tree.*

In [20]:
def uniform_cost_search(env):
    """Uniform cost search"""
    node = Node(env=env,
                parent=None,
                action=None,
                action_id=None,
                path_cost=0)
    if node.state.is_terminal():
        return get_solution(node)
    frontier = queue.PriorityQueue()
    frontier.put((0, id(node), node)) # include the object id in the priority queue. prevents type error when path_costs are identical.
    while True:
        assert frontier, "Failed to find a goal state"
        node = frontier.get()[2]
        if node.state.is_terminal():
            return get_solution(node)
        for action in get_actions(node.state):
            child = get_child_node(node, action)
            frontier.put((child.path_cost, id(child), child))

In [21]:
%%time
path, path_cost = test(uniform_cost_search)
print("Path: {}".format(path))
print("Cost: {}".format(path_cost))

Path: [array([1., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0.])]
Cost: 14120.666698099025
CPU times: user 667 ms, sys: 14.7 ms, total: 682 ms
Wall time: 693 ms


Compared with BFS, this is clearly far preferable in the sense that it does find the least cost path for our problem.